In [2]:
import os
import argparse

from time import time
from datetime import datetime

import pandas as pd
import json
from libcomcat.search import count, get_event_by_id, search

#lon min, lon max, lat min, lat max
#sw lon, ne lon, sw lat, ne lat

In [16]:
co_list = []
for co_obj in co_bb:
    co_list.append(co_obj)

In [ ]:
for co in co_list:
    output_file = f'{co.lower()}_eq.csv'
    download_csv(output_file, dt_start, dt_end, mag_start, mag_end, co, co_bb)

In [55]:
co_list[0]

'ABW'

In [49]:
with open('countries_bbox.json', encoding = 'utf-8') as j:
        co_bb = json.load(j)


In [207]:
country = 'USA'
dt_start = '1990-01-01'
dt_end = '1990-04-01'
mag_start = 4
mag_end = 5
headers = ['event_id', 'location', 'latitude', 'longitude', 'depth', 'magnitude', 'significance', 'alert', 'url', 'eventtype' ,'country']

In [216]:
get_all_countries = [co_obj for co_obj in co_bb]

In [ ]:
get_all_countries

In [61]:
eq_events = []

In [62]:
temp_list = ['USA', 'ABW']

In [182]:
eq_events = []
for country in temp_list:
    mag_events = search(starttime=datetime(int(dt_start[0:4]), int(dt_start[5:7]), int(dt_start[8:11])), 
                        endtime=datetime(int(dt_end[0:4]), int(dt_end[5:7]), int(dt_end[8:11])),
                        minlatitude=co_bb[country]['sw']['lat'], 
                        maxlatitude=co_bb[country]['ne']['lat'], 
                        minlongitude=co_bb[country]['sw']['lon'], 
                        maxlongitude=co_bb[country]['ne']['lon'],
                        minmagnitude=mag_start, maxmagnitude=mag_end)
    eq_events.extend(mag_events)

In [213]:
event_list = []
for events in eq_events:
    events = events.toDict()
    event_list.append([events['id'], events['location'], events['latitude'], events['longitude'],
                 events['depth'], events['magnitude'], events['significance'], events['alert'],
        events['url'], events['eventtype'], country])

df=pd.DataFrame(event_list,columns=headers)

In [215]:
df.dtypes

event_id         object
location         object
latitude        float64
longitude       float64
depth           float64
magnitude       float64
significance      int64
alert            object
url              object
eventtype        object
country          object
dtype: object

In [178]:
df['radius'] = df['radius'] + df['km']

In [219]:
dt_start = '1990-01-01'
dt_end = '1990-04-01'
mag_start = 4
mag_end = 8

In [220]:
def pull_from_api(dt_start, dt_end, mag_start, mag_end, country, co_bb):
    eq_events = []
    mag_events = search(starttime=datetime(int(dt_start[0:4]), int(dt_start[5:7]), int(dt_start[8:11])), 
                        endtime=datetime(int(dt_end[0:4]), int(dt_end[5:7]), int(dt_end[8:11])),
                        minlatitude=co_bb[country]['sw']['lat'], 
                        maxlatitude=co_bb[country]['ne']['lat'], 
                        minlongitude=co_bb[country]['sw']['lon'], 
                        maxlongitude=co_bb[country]['ne']['lon'],
                        minmagnitude=mag_start, maxmagnitude=mag_end)
    eq_events.extend(mag_events)
    return eq_events

In [228]:
event_list = []
eq_events = []
with open('countries_bbox.json', encoding = 'utf-8') as j:
    co_bb = json.load(j)
all_countries = [co_obj for co_obj in co_bb]
for country in all_countries:
    try:
        eq_events = pull_from_api(dt_start, dt_end, mag_start, mag_end, country, co_bb)
    except Exception:
        pass
    print(country)
    for events in eq_events:
        events = events.toDict()
        event_list.append([events['id'], events['location'], events['latitude'], events['longitude'],
                    events['depth'], events['magnitude'], events['significance'], events['alert'],
                    events['url'], events['eventtype'], country])
    
df=pd.DataFrame(event_list,columns=headers)

ABW
AFG
AGO
AIA
ALA
ALB
AND
ANT
ARE
ARG
ARM
ASM
ATA
ATF
ATG
AUS
AUT
AZE
BDI
BEL
BEN
BFA
BGD
BGR
BHR
BHS
BIH
BLM
BLR
BLZ
BMU
BOL
BRA
BRB
BRN
BTN
BVT
BWA
CAF
CAN
CCK
CHE
CHL
CHN
CIV
CMR
COD
COG
COK
COL
COM
CPV
CRI
CUB
CXR
CYM
CYP
CZE
DEU
DJI
DMA
DNK
DOM
DZA
ECU
EGY
ERI
ESH
ESP
EST
ETH
FIN
FJI
FLK
FRA
FRO
FSM
GAB
GBR
GEO
GGY
GHA
GIB
GIN
GLP
GMB
GNB
GNQ
GRC
GRD
GRL
GTM
GUF
GUM
GUY
HKG
HMD
HND
HRV
HTI
HUN
IDN
IMN
IND
IOT
IRL
IRN
IRQ
ISL
ISR
ITA
JAM
JEY
JOR
JPN
KAZ
KEN
KGZ
KHM
KIR
KNA
KOR
KWT
LAO
LBN
LBR
LBY
LCA
LIE
LKA
LSO
LTU
LUX
LVA
MAC
MAF
MAR
MCO
MDA
MDG
MDV
MEX
MHL
MKD
MLI
MLT
MMR
MNE
MNG
MNP
MOZ
MRT
MSR
MTQ
MUS
MWI
MYS
MYT
NAM
NCL
NER
NFK
NGA
NIC
NIU
NLD
NOR
NPL
NRU
NZL
OMN
PAK
PAN
PCN
PER
PHL
PLW
PNG
POL
PRI
PRK
PRT
PRY
PSE
PYF
QAT
REU
ROU
RUS
RWA
SAU
SDN
SEN
SGP
SGS
SHN
SJM
SLB
SLE
SLV
SMR
SOM
SPM
SRB
STP
SUR
SVK
SVN
SWE
SWZ
SYC
SYR
TCA
TCD
TGO
THA
TJK
TKL
TKM
TLS
TON
TTO
TUN
TUR
TUV
TWN
TZA
UGA
UKR
UMI
URY
USA
UZB
VAT
VCT
VEN
VGB
VIR
VNM
VUT
WLF
WSM
YEM
ZAF
ZMB
ZWE


In [235]:
df.to_csv('data/test.csv', encoding='utf-8')


In [229]:
len(df)

1914

In [230]:
df

,event_id,location,latitude,longitude,depth,magnitude,significance,alert,url,eventtype,country
0,usp00043xr,"69 km NNE of P?r?n, Afghanistan",36.006,71.203,79.6,4.6,326,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,AFG
1,usp0004404,"81 km ENE of Khorugh, Tajikistan",37.860,72.353,33.0,4.8,354,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,AFG
2,usp00044fx,"62 km WNW of Murghob, Tajikistan",38.367,73.303,160.4,4.4,298,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,AFG
3,usp00044jf,"10 km E of Jal?l?b?d, Afghanistan",34.419,70.560,33.0,4.5,312,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,AFG
4,usp00044jy,"42 km ESE of ?uk?mat? Azrah, Afghanistan",34.020,70.065,33.0,4.7,340,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,AFG
...,...,...,...,...,...,...,...,...,...,...,...
1909,usp000466r,"156 km NNW of Caluula, Somalia",13.329,50.361,10.0,4.7,340,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,YEM
1910,usp0004678,"133 km NNW of Caluula, Somalia",13.126,50.423,10.0,4.8,354,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,YEM
1911,usp00045eu,"13 km NE of Carletonville, South Africa",-26.286,27.507,5.0,4.5,312,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,ZAF
1912,usp00045zg,"4 km SW of Stilfontein, South Africa",-26.872,26.736,5.0,4.6,326,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,ZAF
